In [1]:
from tools import *

In [2]:
import torch
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import pandas as pd
import timeit
import numpy as np
from typing import Tuple, Callable

In [3]:
BoxDimType = Tuple[float, float]
TensorType = torch.Tensor
UpdateFuncType = Callable[[TensorType], TensorType]


<div dir="rtl" lang="he" xml:lang="he">

# תנועה קולקטיבית - מודלים טובים ומודלים טובים יותר


תנועה קולקטיבית היא אחד התופעות המענייונות ביותר,
קבוצה של בעלי חיים נעה במרחב בצורה כאילו מאורגנת ללא שוב מנהיג נראה לעין. 
אלו מודלים יכולים להסביר את התנועה הזאת?
והאם נמצא דרך להשוות בן המודלים האלו?

<div dir="rtl" lang="he" xml:lang="he">

## מודל פשוט - vicsek
המודל הבסיסי ביותר, הוא המודל של 
vicsek
כל בעל חיים מתואר ע"י מיקומו,
מסומן ב
$\vec{r}_i \in R^3$
ווקטור הכיוון שלו שמסומן ב
$\vec{v}_i \in R^2$
כאשר כלל העידכון הוא הליכה בכיון הממוצע. 
ע"י חישוב וקטור הכיון של השכנים, מתואר ע"י המשוואה 
$$
\\ \vec{S}_i = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}_i}{|\vec{S}_i|}*v_0
$$
כאשר 
$v_0$ 
הוא גודל קבוע המייצג את מהירות בעל החיים. 




$$
\\ \vec{S} = \sum_{j\in \text{Neg}(i)}{\vec{v}_j}
\\ \vec{v}_i = \frac{\vec{S}}{|\vec{S}|}*v_0
$$

In [4]:
def get_avrage_velocity_by_radios(data: TensorType, radios: float = 5.0) -> TensorType:
    """get the avrage velocity of the input data

    Args:
        data (TensorType): tensor (N,5) of x,y,z,theta,phi
        radios (float, optional): radios of . Defaults to 5.0.

    Returns:
        TensorType: tensor (N,2) new velocity of the input data
    """
    # for each data point, get the distance from any other data point
    dis_mat = torch.stack(
        [torch.linalg.norm(data[i, :3] - data[:, :3], dim=1) for i in range(len(data))]
    )
    # for each data point, get the avrage velocity of the data points that are in the radios
    new_velocity = torch.stack(
        [data[dis_mat[i] < radios, 3:].mean(dim=0) for i in range(len(data))]
    )
    return new_velocity


In [35]:
def vicsek_update(
    data: TensorType,
    base_speed: float = 0.1,
    radius: float = 1.0,
    in_place: bool = False,
) -> TensorType:
    new_velocity = get_avrage_velocity(data, radius)
    if in_place:
        data[:, 3:] = new_velocity
        data[:, :3] += base_speed * spherical_to_cartesian(
            torch.ones(data.shape[0]),
            new_velocity[:, 0],
            new_velocity[:, 1],
        )
        return data
    else:
        return torch.column_stack(
            [
                data[:,:3]
                + base_speed
                * spherical_to_cartesian(
                    torch.ones(data.shape[0]),
                    new_velocity[:, 0],
                    new_velocity[:, 1],
                ),
                new_velocity,
            ]
        )

In [46]:
data = torch.rand(200, 5)
data[:, 3:] = data[:, 3:] * 360
data[:, :3] = data[:, :3]
timeline = create_timeline_series(data, vicsek_update, 20, radius=0.2)